In [1]:
!pip install -q arxiv
!pip install -q pymupdf
!pip install -q sentence-transformers
!pip install -q faiss-cpu
!pip install -q evaluate
!pip install -q datasets
!pip install -q bitsandbytes
!pip install -q peft
!pip install -q trl
!pip install -q einops
!pip install -q langchain
!pip install -q langchain-community
!pip install -q rouge_score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 77.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00

In [2]:
import os
import re
import arxiv
import fitz  
import faiss
import numpy as np
from datasets import Dataset
from sentence_transformers import SentenceTransformer
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

2025-09-14 03:37:55.606828: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757821075.926478      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757821076.016136      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# data collection
def get_cvpr_papers(max_results=6):
    search = arxiv.Search(
        query="cat:cs.CV", 
        max_results=max_results,
        sort_by=arxiv.SortCriterion.SubmittedDate
    )
    papers = []
    for result in search.results():
        papers.append({
            'title': result.title,
            'authors': [author.name for author in result.authors],
            'summary': result.summary,
            'published': result.published,
            'pdf_url': result.pdf_url
        })
    return papers

def download_and_extract_text(papers):
    
    for paper in papers:
        try:
            client = arxiv.Client()
            paper_obj = next(client.results(arxiv.Search(id_list=[paper['pdf_url'].split('/')[-1]])))
            pdf_path = paper_obj.download_pdf()
            doc = fitz.open(pdf_path)
            full_text = ""
            for page in doc:
                full_text += page.get_text()
            paper['full_text'] = full_text
        except Exception as e:
            print(f"Failed to process {paper['title']}: {e}")
            paper['full_text'] = ""
    return papers

# preprocessing
def clean_text(text):
    text = re.sub(r'\s+', ' ', text)  
    text = re.sub(r'-\n', '', text)    
    text = text.strip()
    return text

def preprocess_papers(papers):
    for paper in papers:
        paper['full_text'] = clean_text(paper['full_text'])
    return papers

# vector store creation
def create_vector_store(papers, model_name="all-MiniLM-L6-v2"):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    docs = []
    for paper in papers:
        if paper['full_text']:
            chunks = text_splitter.split_text(paper['full_text'])
            for chunk in chunks:
                docs.append({
                    "content": chunk,
                    "metadata": {
                        "title": paper['title'],
                        "authors": ", ".join(paper['authors']),
                        "published": paper['published']
                    }
                })

    # Use LangChain's document structure
    from langchain.docstore.document import Document
    langchain_docs = [Document(page_content=doc['content'], metadata=doc['metadata']) for doc in docs]


    embeddings = HuggingFaceEmbeddings(model_name=model_name)
    vector_store = FAISS.from_documents(langchain_docs, embeddings)
    return vector_store


if __name__ == '__main__':
    # Fetch and process papers
    cvpr_papers = get_cvpr_papers(max_results=10) 
    cvpr_papers_with_text = download_and_extract_text(cvpr_papers)
    preprocessed_papers = preprocess_papers(cvpr_papers_with_text)

    # Create and save the vector store
    vector_store = create_vector_store(preprocessed_papers)
    vector_store.save_local("faiss_index_cvpr")

    print("Vector store created successfully.")

/tmp/ipykernel_36/4061743921.py:9: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  for result in search.results():
/tmp/ipykernel_36/4061743921.py:70: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=model_name)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Vector store created successfully.


In [5]:
import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import pandas as pd
import os

# disable wandb_api
os.environ["WANDB_DISABLED"] = "true"


# a dummy 'preprocessed_papers' object is created if it doesn't exist.
if 'preprocessed_papers' not in locals():
    print("Creating dummy data for 'preprocessed_papers' to ensure code runs.")
    preprocessed_papers = [
        {
            'title': 'Attention Is All You Need',
            'summary': 'The dominant sequence transduction models are based on complex recurrent or convolutional neural networks... We propose a new simple network architecture, the Transformer, based solely on attention mechanisms.',
        },
        {
            'title': 'Generative Adversarial Nets',
            'summary': 'We propose a new framework for estimating generative models via an adversarial process... a generative model G that captures the data distribution, and a discriminative model D that estimates the probability that a sample came from the training data rather than G.',
        }
    ]

# Dataset Preparation (Synthetic Q&A) 
def create_synthetic_dataset(papers):
    prompts = []
    for paper in papers:
        if paper['summary']:
            methodology_prompt = f"### Question: What is the methodology proposed in the paper titled '{paper['title']}'?\n### Answer: {paper['summary']}"
            contribution_prompt = f"### Question: What are the main contributions of '{paper['title']}'?\n### Answer: {paper['summary']}"
            prompts.append(methodology_prompt)
            prompts.append(contribution_prompt)
    return Dataset.from_dict({"text": prompts})

# model configuration
model_id = "Qwen/Qwen3-4B-Instruct-2507"
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# LoRA config
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# fine tuning with transformers.Trainer
if __name__ == '__main__':
    synthetic_dataset = create_synthetic_dataset(preprocessed_papers)

    tokenized_dataset = synthetic_dataset.map(
        lambda examples: tokenizer(examples["text"], truncation=True, max_length=512, padding=False),
        batched=True
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True
    )

    model.config.use_cache = False
    model = prepare_model_for_kbit_training(model)
    model = get_peft_model(model, lora_config)

    training_args = TrainingArguments(
        output_dir="./qwen3-4b-finetuned",
        num_train_epochs=10,
        per_device_train_batch_size=2,
        gradient_accumulation_steps=2,
        logging_dir="./logs",
        logging_steps=10,
        learning_rate=2e-4,
        save_strategy="epoch",
        fp16=True,
        report_to="none",
    )

    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset,
        tokenizer=tokenizer,
        data_collator=data_collator,
    )

    # Start the training process
    trainer.train()

    # Save the fine-tuned model adapters
    trainer.save_model("./qwen3-4b-finetuned")
    tokenizer.save_pretrained("./qwen3-4b-finetuned")
    print("\nFine-tuning complete and model adapters saved successfully.")

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

...

generation_config.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

/tmp/ipykernel_36/1033990311.py:103: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,2.215000
20,1.344700
30,0.508700
40,0.084300
50,0.025900


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/


Fine-tuning complete and model adapters saved successfully.


In [6]:
# merge fine tuned adapters with base model
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import os

# configuration
base_model_id = "Qwen/Qwen3-4B-Instruct-2507"
adapter_path = "./qwen3-4b-finetuned"
merged_model_path = "./qwen3-4b-cvpr-merged" 

print("Loading base model...")
# Load the base model with the same precision as used for training
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
)

print("Loading PEFT adapters...")
# Load the PEFT model (base model + adapters)
model = PeftModel.from_pretrained(base_model, adapter_path)

print("Merging model and adapters...")
# Merge the adapters into the base model
model = model.merge_and_unload()
print("Merge complete.")

print(f"Saving merged model to {merged_model_path}...")
# Save the merged model and its tokenizer
model.save_pretrained(merged_model_path)
tokenizer = AutoTokenizer.from_pretrained(adapter_path)
tokenizer.save_pretrained(merged_model_path)

print("\nModel merged and saved successfully!")
print(f"You can now download the '{merged_model_path}' directory and use it with Ollama.")

# List files to confirm
print("\nFiles in the new merged model directory:")
for dirname, _, filenames in os.walk(merged_model_path):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Loading base model...


`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loading PEFT adapters...
Merging model and adapters...
Merge complete.
Saving merged model to ./qwen3-4b-cvpr-merged...

Model merged and saved successfully!
You can now download the './qwen3-4b-cvpr-merged' directory and use it with Ollama.

Files in the new merged model directory:
./qwen3-4b-cvpr-merged/model-00001-of-00002.safetensors
./qwen3-4b-cvpr-merged/special_tokens_map.json
./qwen3-4b-cvpr-merged/chat_template.jinja
./qwen3-4b-cvpr-merged/model-00002-of-00002.safetensors
./qwen3-4b-cvpr-merged/config.json
./qwen3-4b-cvpr-merged/tokenizer.json
./qwen3-4b-cvpr-merged/vocab.json
./qwen3-4b-cvpr-merged/generation_config.json
./qwen3-4b-cvpr-merged/model.safetensors.index.json
./qwen3-4b-cvpr-merged/merges.txt
./qwen3-4b-cvpr-merged/tokenizer_config.json
./qwen3-4b-cvpr-merged/added_tokens.json


In [7]:
import shutil
# config
folder_to_zip = 'qwen3-4b-cvpr-merged'
output_zip_filename = 'qwen3-4b-cvpr-merged' # The .zip extension will be added automatically
if os.path.isdir(folder_to_zip):
    print(f"Zipping the directory: '{folder_to_zip}'...")
    shutil.make_archive(
        base_name=output_zip_filename,  
        format='zip',                   
        root_dir=folder_to_zip          
    )
    print(f"\nSuccessfully created '{output_zip_filename}.zip'")
    print("You can now find this file in the file explorer panel on the left and download it.")
else:
    print(f"Error: The directory '{folder_to_zip}' does not exist.")
    print("Please make sure you have run the model merging script successfully.")

Zipping the directory: 'qwen3-4b-cvpr-merged'...

Successfully created 'qwen3-4b-cvpr-merged.zip'
You can now find this file in the file explorer panel on the left and download it.


In [1]:
# RAG pipeline with local model in Kaggle
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

print("Loading RAG components with a local model...")

# Load FAISS vector score
embeddings_model_name = "all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=embeddings_model_name)
faiss_index_path = "/kaggle/working/faiss_index_cvpr"
vector_store = FAISS.load_local(
    faiss_index_path,
    embeddings,
    allow_dangerous_deserialization=True
)
print(f"FAISS index loaded successfully from: {faiss_index_path}")

# Load local fine tuned model and tokenizer
local_model_path = "/kaggle/working/qwen3-4b-cvpr-merged"
print(f"Loading tokenizer from: {local_model_path}")
tokenizer = AutoTokenizer.from_pretrained(local_model_path)
print(f"Loading model from: {local_model_path}")
model = AutoModelForCausalLM.from_pretrained(
    local_model_path,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)
print("Model loaded successfully onto the GPU.")

# Transformers pipeline for text generation
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.1,
    top_p=0.95
)
local_llm = HuggingFacePipeline(pipeline=pipe)
print("LangChain LLM wrapper created successfully.")

prompt_template = """Context: {context}

Question: {question}

Answer:"""
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

# create RAG chain
qa_chain = RetrievalQA.from_chain_type(
    llm=local_llm,
    chain_type="stuff",
    retriever=vector_store.as_retriever(search_kwargs={"k": 3}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)
print("RAG chain created successfully with simplified prompt.")

# Querying the RAG pipeline
if __name__ == '__main__':
    query = "What is the name of the new benchmark for measuring epistemic humility in MLLMs?"
    print(f"\nTesting RAG chain with query: '{query}'")
    try:
        result = qa_chain({"query": query})
        print("\n--- Test RAG Response ---")
        print(result['result'])
    except Exception as e:
        print(f"\nAn error occurred while querying the RAG chain: {e}")

2025-09-14 05:03:46.201705: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757826226.564787     298 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757826226.672271     298 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Loading RAG components with a local model...


/tmp/ipykernel_298/3557732259.py:14: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=embeddings_model_name)


FAISS index loaded successfully from: /kaggle/working/faiss_index_cvpr
Loading tokenizer from: /kaggle/working/qwen3-4b-cvpr-merged


`torch_dtype` is deprecated! Use `dtype` instead!


...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


Model loaded successfully onto the GPU.
LangChain LLM wrapper created successfully.
RAG chain created successfully with simplified prompt.

Testing RAG chain with query: 'What is the name of the new benchmark for measuring epistemic humility in MLLMs?'


/tmp/ipykernel_298/3557732259.py:44: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  local_llm = HuggingFacePipeline(pipeline=pipe)
/tmp/ipykernel_298/3557732259.py:69: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": query})



--- Test RAG Response ---
Context: Finally, we conduct a manual filtering process to refine the questions and options. are named HumbleBench, HumbleBench-E, and HumbleBench-GN, respectively. HumbleBench is the original benchmark. HumbleBench-E and HumbleBench-GN are two stress tests. Specif- ically, HumbleBench-E removes correct non-E answers so the E option, i.e., “None of the above”, is the only correct answer for the entire dataset. HumbleBench-GN replaces all images with Gaus- sian noise so, again, only the E option is correct. Accuracy (%) is reported as the primary metric throughout the experiments. 3.1 Results on HumbleBench Overall Accuracy Well Above Random Guess But Far From Perfect. As shown in Table 3, HumbleBench presents a considerable challenge for current state-of-the-art MLLMs. The 5 Table 3: Performance of general-purpose and specialized reasoning models on HumbleBench. The best- performing model in each category is highlighted in bold. Model # Params Object Relation

In [2]:
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

print("Initializing agent with the local LLM...")


tools = [
    Tool(
        name="CVPR Research Paper QA System",
        func=qa_chain, # The function of the tool is our RAG chain
        description=(
            "Use this tool to answer specific questions about computer vision research papers from CVPR. "
            "The input must be a complete, well-formed question."
        )
    )
]

agent = initialize_agent(
    tools,
    local_llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True, # Set to True to see the agent's thought process
    handle_parsing_errors=True # Helps with occasional model formatting issues
)
print("Agent initialized successfully.")


if __name__ == '__main__':
    agent_query = (
        "Find a paper on generative models and explain its core methodology based on the provided context."
    )
    print(f"\nExecuting agent with query: '{agent_query}'")
    try:
        # The agent's 'run' method invokes the thought-action-observation loop
        response = agent.run(agent_query)
        print("\n--- Agent's Final Answer ---")
        print(response)
    except Exception as e:
        print(f"\nAn error occurred while running the agent: {e}")

Initializing agent with the local LLM...
Agent initialized successfully.

Executing agent with query: 'Find a paper on generative models and explain its core methodology based on the provided context.'


> Entering new AgentExecutor chain...


/tmp/ipykernel_298/2754730473.py:18: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(
/tmp/ipykernel_298/2754730473.py:35: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent.run(agent_query)


Parsing LLM output produced both a final answer and a parse-able action:: Answer the following questions as best you can. You have access to the following tools:

CVPR Research Paper QA System(inputs: Union[dict[str, Any], Any], return_only_outputs: bool = False, callbacks: Union[list[langchain_core.callbacks.base.BaseCallbackHandler], langchain_core.callbacks.base.BaseCallbackManager, NoneType] = None, *, tags: Optional[list[str]] = None, metadata: Optional[dict[str, Any]] = None, run_name: Optional[str] = None, include_run_info: bool = False) -> dict[str, typing.Any] - Use this tool to answer specific questions about computer vision research papers from CVPR. The input must be a complete, well-formed question.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [CVPR Research Paper QA System]
Action Input: the input to the action
Observation: the result of the action
..

In [3]:
# Evaluation Metrics

import pandas as pd
from evaluate import load
from sentence_transformers import SentenceTransformer, util
import numpy as np

print("Starting RAG pipeline evaluation with the simplified prompt...")

# a small but accurate evaluation dataset is created through preprocessed papers
eval_dataset = [
    {
        "question": "What is the name of the framework proposed for graph alignment and what does it stand for?",
        "ground_truth_answer": "The paper introduces GADL, which stands for Graph Alignment with Dual-pass encoder and Latent space communication.",
        "ground_truth_context_substring": "In this paper, we introduce GADL—Graph Alignment with Dual-pass encoder and Latent space communication"
    },
    {
        "question": "What are the two critical limitations of existing unsupervised graph alignment methods?",
        "ground_truth_answer": "They suffer from the degradation of node distinctiveness due to GNN oversmoothing and the misalignment of latent spaces across different graphs.",
        "ground_truth_context_substring": "suffer from two critical limitations: the degradation of node distinctiveness due to oversmoothing in GNN-based embeddings, and the misalignment of latent spaces"
    },
    {
        "question": "What is the name of the new benchmark for measuring epistemic humility in MLLMs?",
        "ground_truth_answer": "The paper presents HumbleBench, a new hallucination benchmark designed to evaluate an MLLM's ability to reject plausible but incorrect answers.",
        "ground_truth_context_substring": "We present HumbleBench, a new hallucination benchmark designed to evaluate MLLMs' ability to reject plausible but incorrect answers"
    },
    {
        "question": "What type of model does the 'Mechanistic Learning' paper propose for predicting brain tumor growth?",
        "ground_truth_answer": "The paper proposes a hybrid mechanistic learning framework that combines a mathematical tumor growth model with a guided denoising diffusion implicit model (DDIM).",
        "ground_truth_context_substring": "propose a hybrid mechanistic learning framework that combines a mathematical tumor growth model with a guided denoising diffusion implicit model (DDIM)"
    },
    {
        "question": "What is the key innovation of the DiFlow-TTS model for speech synthesis?",
        "ground_truth_answer": "DiFlow-TTS is the first model to explore a purely Discrete Flow Matching approach for speech synthesis, unlike previous methods that used continuous spaces.",
        "ground_truth_context_substring": "we introduce DiFlow-TTS, which, to the best of our knowledge, is the first model to explore purely Discrete Flow Matching for speech synthesis"
    },
    {
        "question": "What are the two primary contributions of the paper on million-scale text-to-image reasoning?",
        "ground_truth_answer": "The paper introduces FLUX-Reason-6M, a massive dataset for reasoning, and PRISM-Bench, a comprehensive evaluation benchmark for text-to-image models.",
        "ground_truth_context_substring": "To address this challenge, We introduce FLUX-Reason-6M and PRISM-Bench"
    }
]

print(f"Evaluation dataset created with {len(eval_dataset)} examples.")

results = []
retriever = vector_store.as_retriever(search_kwargs={'k': 5})

for item in eval_dataset:
    question = item["question"]
    ground_truth_context_sub = item["ground_truth_context_substring"]
    retrieved_docs = retriever.get_relevant_documents(question)
    retrieved_contexts = [doc.page_content for doc in retrieved_docs]
    hit = any(ground_truth_context_sub.lower() in ctx.lower() for ctx in retrieved_contexts)
    rr = 0.0
    for i, ctx in enumerate(retrieved_contexts):
        if ground_truth_context_sub.lower() in ctx.lower():
            rr = 1 / (i + 1)
            break
    response = qa_chain({"query": question})
    generated_answer = response['result']
    results.append({
        "question": question,
        "generated_answer": generated_answer,
        "retrieved_contexts": retrieved_contexts,
        "hit": hit,
        "rr": rr
    })

results_df = pd.DataFrame(results)
print("Evaluation loop complete.")

# calculate and report metrics
print("\n--- Evaluation Results ---")

hit_rate = results_df['hit'].mean()
mrr = results_df['rr'].mean()

print(f"\n--- Retriever Performance ---")
print(f"Hit Rate (Top 5): {hit_rate:.2%}")
print("  => What percentage of the time was the correct context found in the top 5 retrieved documents?")
print(f"Mean Reciprocal Rank (MRR): {mrr:.4f}")
print("  => On average, how high up was the correct document in the retrieved list? (Closer to 1 is better)")
rouge = load('rouge')
semantic_model = SentenceTransformer('all-MiniLM-L6-v2')
ground_truth_answers = [item["ground_truth_answer"] for item in eval_dataset]
generated_answers = results_df['generated_answer'].tolist()
rouge_scores = rouge.compute(predictions=generated_answers, references=ground_truth_answers)
gt_embeddings = semantic_model.encode(ground_truth_answers, convert_to_tensor=True)
gen_embeddings = semantic_model.encode(generated_answers, convert_to_tensor=True)
cosine_scores = util.cos_sim(gt_embeddings, gen_embeddings)
mean_semantic_similarity = np.diag(cosine_scores.cpu()).mean()

print(f"\n--- Generator Performance ---")
print(f"ROUGE-L Score: {rouge_scores['rougeL']:.4f}")
print("  => Measures the longest common subsequence between generated and ground truth answers (word overlap).")
print(f"Mean Semantic Similarity: {mean_semantic_similarity:.4f}")
print("  => Measures if the *meaning* of the generated answer is close to the ground truth (closer to 1 is better).")
print("\n--- Detailed Per-Question Results ---")
for index, row in results_df.iterrows():
    print(f"\nQuestion: {row['question']}")
    print(f"  Ground Truth: {ground_truth_answers[index]}")
    print(f"  Generated: {row['generated_answer']}")
    print(f"  Context Hit: {'Yes' if row['hit'] else 'No'}")
    print(f"  Reciprocal Rank: {row['rr']:.2f}")

Starting RAG pipeline evaluation with the simplified prompt...
Evaluation dataset created with 6 examples.


/tmp/ipykernel_298/4031972433.py:52: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(question)
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Evaluation loop complete.

--- Evaluation Results ---

--- Retriever Performance ---
Hit Rate (Top 5): 50.00%
  => What percentage of the time was the correct context found in the top 5 retrieved documents?
Mean Reciprocal Rank (MRR): 0.1583
  => On average, how high up was the correct document in the retrieved list? (Closer to 1 is better)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


--- Generator Performance ---
ROUGE-L Score: 0.0320
  => Measures the longest common subsequence between generated and ground truth answers (word overlap).
Mean Semantic Similarity: 0.5759
  => Measures if the *meaning* of the generated answer is close to the ground truth (closer to 1 is better).

--- Detailed Per-Question Results ---

Question: What is the name of the framework proposed for graph alignment and what does it stand for?
  Ground Truth: The paper introduces GADL, which stands for Graph Alignment with Dual-pass encoder and Latent space communication.
  Generated: Context: inconsistencies and challenging alignment scenarios. Additionally, compre- hensive evaluation on vision-language benchmarks using diverse pretrained models shows that our framework effectively generalizes beyond graph domains, enabling unsupervised alignment of vision and language representations. 1 Introduction Graph alignment—also referred to as network alignment or graph matching—is a fundamental prob

In [19]:
import subprocess
import pandas as pd

def gpu_process_report():
    # Get GPU processes from nvidia-smi
    nvidia_out = subprocess.check_output(
        "nvidia-smi --query-compute-apps=pid,process_name,used_memory --format=csv,noheader,nounits",
        shell=True
    ).decode().strip().split("\n")

    processes = []
    for line in nvidia_out:
        if line.strip():
            pid, name, mem = [x.strip() for x in line.split(",")]
            processes.append((int(pid), name, int(mem)))

    # Get details from ps for those PIDs
    ps_out = subprocess.check_output("ps -eo pid,ppid,cmd --no-headers", shell=True).decode().splitlines()
    ps_map = {int(x.split(None, 2)[0]): x for x in ps_out}

    # Build table
    rows = []
    for pid, name, mem in processes:
        ps_info = ps_map.get(pid, "")
        rows.append({
            "PID": pid,
            "GPU Memory (MiB)": mem,
            "Process Name (from nvidia-smi)": name,
            "PS Info": ps_info
        })

    df = pd.DataFrame(rows)
    if df.empty:
        print("✅ No processes currently using the GPU")
    else:
        from IPython.display import display
        display(df)

gpu_process_report()


,PID,GPU Memory (MiB),Process Name (from nvidia-smi),PS Info
0,4128,14898,[Not Found],
1,4128,15082,[Not Found],


In [15]:
import gc, torch

# Delete references to model + tokenizer
del model
del tokenizer

gc.collect()
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()
torch.cuda.synchronize()

print("✅ Model deleted and VRAM cache cleared")


✅ Model deleted and VRAM cache cleared


In [ ]:
!kill -9 36